In [1]:
task = "ner" #需要是"ner", "pos" 或者 "chunk"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

# 导入库

In [34]:
import torch
from datasets import load_dataset, load_metric
import transformers
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import TrainingArguments, Trainer

# 加载数据

In [4]:
datasets = load_dataset("conll2003")

Downloading:   0%|          | 0.00/2.60k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.78k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/650k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/163k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/146k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset conll2003 downloaded and prepared to C:\Users\lyk\.cache\huggingface\datasets\conll2003\conll2003\1.0.0\40e7cb6bcc374f7c349c83acd1e9352a4f09474eb691f64f364ee62eb65d0ca6. Subsequent calls will reuse this data.


In [5]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [9]:
datasets['train'][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [10]:
datasets['train'].features['ner_tags']

Sequence(feature=ClassLabel(num_classes=9, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], names_file=None, id=None), length=-1, id=None)

In [13]:
label_list = datasets['train'].features[f"{task}_tags"].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [18]:
# 为了能够进一步理解数据长什么样子，随机选择数据进行展示
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(datasets, num_examples=10):
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(datasets) -1)
        while pick in picks:
            pick = random.randint(0, len(datasets)-1)
        picks.append(pick)    
    
    df = pd.DataFrame(datasets[picks])
    for column, typ in datasets.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i:typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [19]:
show_random_elements(datasets['train'])

,id,tokens,pos_tags,chunk_tags,ner_tags
0,522,"[Scorers, :]","[NNS, :]","[B-NP, O]","[O, O]"
1,10046,"[6., Jans, Koerts, (, Netherlands, ), Palmans]","[CD, NNPS, NNP, (, NNP, ), NNS]","[B-NP, I-NP, I-NP, I-NP, I-NP, O, B-NP]","[O, B-PER, I-PER, O, B-LOC, O, B-ORG]"
2,4945,"[West, Ham, 3, 1, 1, 1, 3, 4, 4]","[NNP, NNP, CD, CD, CD, CD, CD, CD, CD]","[B-NP, I-NP, I-NP, I-NP, I-NP, I-NP, I-NP, I-NP, I-NP]","[B-ORG, I-ORG, O, O, O, O, O, O, O]"
3,12,"[Only, France, and, Britain, backed, Fischler, 's, proposal, .]","[RB, NNP, CC, NNP, VBD, NNP, POS, NN, .]","[B-NP, I-NP, I-NP, I-NP, B-VP, B-NP, B-NP, I-NP, O]","[O, B-LOC, O, B-LOC, O, B-PER, O, O, O]"
4,288,"[Trade, balance, (, million, markka, ), :]","[NN, NN, (, CD, NNS, ), :]","[B-NP, I-NP, O, B-NP, I-NP, O, O]","[O, O, O, O, O, O, O]"
5,8694,"[Hamburg, 0, VfB, Stuttgart, 4]","[NNP, CD, NNP, NNP, CD]","[B-NP, I-NP, I-NP, I-NP, I-NP]","[B-ORG, O, B-ORG, I-ORG, O]"
6,5793,"[SYDNEY, 1996-08-26]","[NNP, CD]","[B-NP, I-NP]","[B-LOC, O]"
7,79,"[AMT, $, 300, MLN, SPREAD, -, 12.5, BP, MATURITY, 21.JAN.99]","[NNP, $, CD, NNP, NNP, :, CD, NNP, NNP, CD]","[O, B-NP, I-NP, I-NP, I-NP, O, B-NP, I-NP, I-NP, I-NP]","[O, O, O, O, O, O, O, O, O, O]"
8,10757,"[1-0, .]","[CD, .]","[B-NP, O]","[O, O]"
9,2365,"[2., Samuel, Matete, (, Zambia, ), 47.99]","[NNP, NNP, NNP, (, NNP, ), CD]","[B-NP, I-NP, I-NP, O, B-NP, O, B-NP]","[O, B-PER, I-PER, O, B-LOC, O, O]"


# 预处理数据

In [25]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [26]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [101, 7592, 1010, 2023, 2003, 2028, 6251, 999, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [30]:
# 由于word会被切分为subtokens,所以要进行处理，使subtokens 与标注对齐
example = datasets['train'][4]
tokenized_input = tokenizer(example['tokens'], is_split_into_words=True)
print(len(example[f"{task}_tags"]), len(tokenized_input['input_ids']))
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]  # 特殊标记的label设为-100,会被忽略
print(len(aligned_labels), len(tokenized_input["input_ids"]))

31 39
39 39


In [31]:
# 我们有两种对齐label的方式：
# 多个subtokens对齐一个word，对齐一个label
# 多个subtokens的第一个subtoken对齐word，对齐一个label，其他subtokens直接赋予-100.
label_all_tokens = True
def tokenize_and_align_labels(examples):
    tokenized_input = tokenizer(examples['tokens'], truncation=True, is_split_into_words=True)
    
    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_input.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_input['labels'] = labels
    return tokenized_input

In [32]:
tokenize_and_align_labels(datasets['train'][:5])

{'input_ids': [[101, 7327, 19164, 2446, 2655, 2000, 17757, 2329, 12559, 1012, 102], [101, 2848, 13934, 102], [101, 9371, 2727, 1011, 5511, 1011, 2570, 102], [101, 1996, 2647, 3222, 2056, 2006, 9432, 2009, 18335, 2007, 2446, 6040, 2000, 10390, 2000, 18454, 2078, 2329, 12559, 2127, 6529, 5646, 3251, 5506, 11190, 4295, 2064, 2022, 11860, 2000, 8351, 1012, 102], [101, 2762, 1005, 1055, 4387, 2000, 1996, 2647, 2586, 1005, 1055, 15651, 2837, 14121, 1062, 9328, 5804, 2056, 2006, 9317, 10390, 2323, 4965, 8351, 4168, 4017, 2013, 3032, 2060, 2084, 3725, 2127, 1996, 4045, 6040, 2001, 24509, 1012, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, -100], [-100, 1, 2, -100], [-100, 5, 0, 

In [33]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

# 微调预训练模型

In [36]:
# 由于我们微调的任务是token分类任务，而我们加载的是预训练的语言模型，所以会提示我们加载模型的时候扔掉了一些不匹配的神经网络参数
# （比如：预训练语言模型的神经网络head被扔掉了，同时随机初始化了token分类的神经网络head）。
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN t

In [46]:
args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_strategy='no',
    report_to='none'
)

In [40]:
from transformers import DataCollatorForTokenClassification

In [41]:
# 数据收集器，用于将处理好的数据输入给模型
data_collator = DataCollatorForTokenClassification(tokenizer)

# 设定评估模型

In [42]:
metric = load_metric("seqeval")

Downloading:   0%|          | 0.00/2.48k [00:00<?, ?B/s]

In [43]:
labels = [label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'LOC': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'PER': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [44]:
import numpy as np

def compute_metrics(p):
    """模型预测"""
    predictions, labels = p
    # 选择预测分类最大概率的下标
    predictions = np.argmax(predictions, axis=2)

    # 将下标转化为label，并忽略-100的位置
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }


In [47]:
# 构造训练器Trainer
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


# 模型训练

In [48]:
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

# 模型评估

In [ ]:
trainer.evaluate()

# 输出单个类别的precision/recall/f1

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results
